# Creazione matrici png

In [2]:
import PIL
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
import random,string
import os, sys, time

# font = ImageFont.truetype("Arial-Bold.ttf",14)
font = ImageFont.truetype("arial.ttf", 100)

N_SAMPLES = 5000 #numero captcha completi
N_SMP_LETTERE_DEFORMATE =  26000
if not os.path.exists("./matrici/"):
    os.makedirs("./matrici/")
if not os.path.exists("./matrici/originale/"):
    os.makedirs("./matrici/originale/")
if not os.path.exists("./captcha/completi/"):
    os.makedirs("./captcha/completi/")
if not os.path.exists("./captcha/singoli/"):
    os.makedirs("./captcha/singoli/")

In [3]:
size = 200
def crea(lettera):
    img=Image.new("1", (size,size),color=1)
    draw = ImageDraw.Draw(img)
    draw.text((50, 50),lettera,fill=0,font=font)
    draw = ImageDraw.Draw(img)
    #noisy(img,size)
    img.save(f"./matrici/originale/{lettera}.png")
    


In [4]:
def noisy(img,width,height):
    draw = ImageDraw.Draw(img)
    for x in range(0,15):
        draw.point((random.randint(0,width),random.randint(0,height)),0)
        draw.line([(random.randint(0,width),random.randint(0,height)), (random.randint(0,width),random.randint(0,height))],width=3)

    return img

# Funzione di noise per i campioni

In [5]:
#creazione matrici
alfabeto = list(string.ascii_uppercase) #or list(map(chr, range(ord('a'), ord('z')+1)))

#alfabeto = ["A","B","C","D","E"]
for l in alfabeto:
    crea(l)

In [6]:
import Augmentor
p = Augmentor.Pipeline("./matrici/originale/")

Initialised with 26 image(s) found.
Output directory set to ./matrici/originale/output.

In [7]:
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)

p.random_distortion(probability=1, grid_width=5, grid_height=5, magnitude=8)

p.skew(probability=1)

In [ ]:
p.sample(N_SMP_LETTERE_DEFORMATE)

Processing <PIL.Image.Image image mode=1 size=200x200 at 0x82FD0F0>:   6%| | 1684/26000 [01:26<11:57, 33.89 Samples/s]  

# Funzione di add noise per i campioni

In [ ]:
def sporca(path):
    #t1=round(time.time() * 1000)

    dirs = os.listdir( path )

    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            im = noisy(im,width=1000,height=200)
            im.save(f"{f}.png")
    #t2=round(time.time() * 1000)
   # print(f"{t2-t1} millisecondi") #5ms a immagine

In [ ]:
import string
def getfirstUppercase(s):
    return ([char for char in s if char.isupper()])[0]

# generazione di captcha random

In [ ]:
path = "./matrici/originale/output/"
dirs = os.listdir( path )

total_width = 1000
max_height = 200
for v in range (0,N_SAMPLES):
    new_im = Image.new("1", (total_width, max_height))
    filename = ""
    x_offset = 0
    list_images = []
    for i in range (0,5): #5 elementi
        curr_rand=random.randint(0,len(dirs)-1)
        list_images.append(path+dirs[curr_rand])
        valore = dirs[curr_rand]
        filename+=getfirstUppercase(valore)

    for im in list_images:
        image_cur = Image.open(im)
        new_im.paste(image_cur, (x_offset,0))
        x_offset += image_cur.size[0]

    new_im.save(f'./captcha/completi/{filename}{v}.png') # es stringa AEDTG45.png

In [ ]:
#aggiunge il noise ai captcha
sporca("./captcha/completi/")

In [ ]:
path = "./captcha/completi/"
dirs = os.listdir( path )

for item in dirs:

    if os.path.isfile(path+item):
        for i in range(0,5):
            im = Image.open(path+item)
            #crop ogni 200px 5 volte
            #0,0 200,200 - 200,0 400,200 - 400,0 600, 200
            current_box = im.crop((200*i,0,200*(i+1),200))

            #legge la lettera corrispondente
            if not os.path.exists("./captcha/singoli/"+item[i]):
                os.makedirs("./captcha/singoli/"+item[i])
            #salva nella cartella corrispondente
            current_box.save(f"./captcha/singoli/{item[i]}/{i}{time.time()}.png")
            
